# Title

TODO: write up about project

Notes on installation?
https://github.com/stefan-jansen/machine-learning-for-trading/tree/main/installation

### Import libraries

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import zipline
from zipline.api import order_percent, order_target, order_value, order, record, symbol
from zipline.finance import commission, slippage
from zipline import run_algorithm

%load_ext zipline
%matplotlib inline

### Download stock data

In [2]:
start_date='2020-01-02'
end_date='2020-12-30'

start_year=start_date[0:4]

In [11]:
def download_stock_data(ticker, start_date, end_date):
    df = yf.download(ticker, start=start_date, end=end_date)
    df = df.reset_index()
    df = df.drop(['Adj Close'], axis=1)
    df = df.rename(str.lower, axis='columns')
    df['dividend'] = 0
    df['split'] = 1
    return df

In [12]:
df_tsla = download_stock_data('TSLA', start_date, end_date)
df_aapl = download_stock_data('AAPL', start_date, end_date)
df_nflx = download_stock_data('NFLX', start_date, end_date)
df_hsbc = download_stock_data('HSBC', start_date, end_date)
df_gs = download_stock_data('GS', start_date, end_date)
df_gme = download_stock_data('GME', start_date, end_date)
df_ge = download_stock_data('GE', start_date, end_date)
df_lyg = download_stock_data('LYG', start_date, end_date)
df_shldq = download_stock_data('SHLDQ', start_date, end_date)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
df_tsla

,date,open,high,low,close,volume,dividend,split
0,2020-01-02,84.900002,86.139999,84.342003,86.052002,47660500,0,1
1,2020-01-03,88.099998,90.800003,87.384003,88.601997,88892500,0,1
2,2020-01-06,88.094002,90.311996,88.000000,90.307999,50665000,0,1
3,2020-01-07,92.279999,94.325996,90.671997,93.811996,89410500,0,1
4,2020-01-08,94.739998,99.697998,93.646004,98.428001,155721500,0,1
...,...,...,...,...,...,...,...,...
247,2020-12-23,632.200012,651.500000,622.570007,645.979980,33173000,0,1
248,2020-12-24,642.989990,666.090027,641.000000,661.770020,22865600,0,1
249,2020-12-28,674.510010,681.400024,660.799988,663.690002,32278600,0,1
250,2020-12-29,661.000000,669.900024,655.000000,665.989990,22910800,0,1


### Save data to CSV file

Note: the CSV file must be the ticker symbol without additional text like `tsla-2011.csv`

In [15]:
df_tsla.to_csv(f'daily/{start_year}/tsla.csv', index=False)
df_aapl.to_csv(f'daily/{start_year}/aapl.csv', index=False)
df_nflx.to_csv(f'daily/{start_year}/nflx.csv', index=False)
df_hsbc.to_csv(f'daily/{start_year}/hsbc.csv', index=False)
df_gs.to_csv(f'daily/{start_year}/gs.csv', index=False)
df_gme.to_csv(f'daily/{start_year}/gme.csv', index=False)
df_ge.to_csv(f'daily/{start_year}/ge.csv', index=False)
df_lyg.to_csv(f'daily/{start_year}/lyg.csv', index=False)
df_shldq.to_csv(f'daily/{start_year}/shldq.csv', index=False)

### Ingest data
Edit ~/.zipline/extension.py according to zipline docs https://zipline.ml4trading.io/bundles.html#ingesting-data-from-csv-files

Note: Zipline seems to handle one custom bundle at a time. Ingest data one by one.

In [10]:
# !zipline ingest -b tsla-2020
# !zipline ingest -b aapl-2020
# !zipline ingest -b nflx-2020
# !zipline ingest -b hsbc-2020
# !zipline ingest -b gs-2020
# !zipline ingest -b gme-2020
# !zipline ingest -b ge-2020
# !zipline ingest -b lyg-2020
!zipline ingest -b shldq-2020

[2021-11-08 02:36:32.716123] INFO: zipline.data.bundles.core: Ingesting shldq-2020.
Loading custom pricing data:   [------------------------------------]    0% | aapl: sid 0
Loading custom pricing data:   [####--------------------------------]   11% | ge: sid 1
Loading custom pricing data:   [########----------------------------]   22% | gme: sid 2
Loading custom pricing data:   [############------------------------]   33% | gs: sid 3
Loading custom pricing data:   [################--------------------]   44% | hsbc: sid 4
Loading custom pricing data:   [####################----------------]   55% | lyg: sid 5
Loading custom pricing data:   [########################------------]   66% | nflx: sid 6
Loading custom pricing data:   [############################--------]   77% | shldq: sid 7
Loading custom pricing data:   [################################----]   88% | tsla: sid 8
Loading custom pricing data:   [####################################]  100%
Merging daily equity files:  [#####

In [12]:
!zipline bundles

csvdir <no ingestions>
gs-2017 2021-11-08 01:51:55.139794
quandl 2021-10-29 00:52:00.736804
quandl 2021-10-29 00:42:42.964339
quandl 2021-10-28 00:23:28.895431
quantopian-quandl <no ingestions>


## Backtesting Trading Strategies

In [3]:
start = pd.Timestamp(start_date).tz_localize("UTC")
end =pd.Timestamp(end_date).tz_localize("UTC")
principal = 10000

In [4]:
# ticker = 'TSLA'
# ticker = 'AAPL'
# ticker = 'NFLX'
# ticker = 'HSBC'
# ticker = 'GS'
# ticker = 'GME'
# ticker = 'GE'
# ticker = 'LYG'
ticker = 'SHLDQ'
bundle_name = f'{ticker.lower()}-{start_year}'

In [5]:
# plotting function
def plot(title, results, columns_to_plot):
    fig, ax = plt.subplots(3, 1, sharex=True, figsize=[16, 9])

    # portfolio value
    results.portfolio_value.plot(ax=ax[0])
    ax[0].set_ylabel('portfolio value in $')

    # asset
    results[columns_to_plot].plot(ax=ax[1])
    ax[1].set_ylabel('price in $')

    # mark transactions
    perf_trans = results.loc[[t != [] for t in results.transactions]]
    buys = perf_trans.loc[[t[0]['amount'] > 0 for t in perf_trans.transactions]]
    sells = perf_trans.loc[[t[0]['amount'] < 0 for t in perf_trans.transactions]]
    ax[1].plot(buys.index, results.price.loc[buys.index], '^', markersize=10, color='g', label='buy')
    ax[1].plot(sells.index, results.price.loc[sells.index], 'v', markersize=10, color='r', label='sell')
    ax[1].legend()

    # daily returns
    results.returns.plot(ax=ax[2])
    ax[2].set_ylabel('daily returns')

    fig.suptitle(title, fontsize=16)
    plt.legend()
    plt.show()

    print('Final portfolio value (including cash): ${}'.format(np.round(results.portfolio_value[-1], 2)))


### Buy and hold

In [5]:
def initialize(context):
    context.asset = symbol(ticker)
    context.has_ordered = False  

def handle_data(context, data):
    record(price=data.current(context.asset, 'price'))
    
    # trading logic
    if not context.has_ordered:
        order_value(context.asset, principal)
        context.has_ordered = True
        
bah_results = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=principal,
    bundle=bundle_name,
    data_frequency="daily",
)

# plot('Buy and Hold - Tesla', bah_results, ['price'])

### Dollar cost averaging


In [6]:
amount_to_invest_weekly = principal / (3*52)

def initialize(context):
    context.days = 0
    context.asset = symbol(ticker)
    context.amount_to_invest_weekly = amount_to_invest_weekly
    context.amont_invested = 0

def handle_data(context, data):
    context.days += 1

    if context.days == 7:
        context.days = 0
        return
    if context.days == 1:
        n_stocks_to_buy = math.floor(context.amount_to_invest_weekly / data.current(context.asset, 'price'))
        if n_stocks_to_buy == 0:
            context.amount_to_invest_weekly += amount_to_invest_weekly
        else:
            context.amont_invested += n_stocks_to_buy * data.current(context.asset, 'price')
            if context.amont_invested < principal:
                order(context.asset, n_stocks_to_buy)
                context.amount_to_invest_weekly = amount_to_invest_weekly
                
    record(price=data.current(context.asset, "price"))
            

dca_results = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=principal,
    bundle=bundle_name,
    data_frequency="daily",
)

# plot('Dollar Cost Averaging - Tesla', dca_results, ['price'])


### Simple moving average

In [7]:
look_back_days = 20

def initialize(context):
    context.days = 0
    context.asset = symbol(ticker)

def handle_data(context, data):
    context.days += 1
    if context.days < look_back_days:
        return

    price_history = data.history(context.asset, fields="price", bar_count=look_back_days, frequency="1d")
 
    # calculate moving averages
    ma = price_history.mean()
    
    # trading logic
    
    # cross up
    if (price_history[-2] < ma) & (price_history[-1] > ma):
        order_percent(context.asset, 1.0)
    # cross down
    elif (price_history[-2] > ma) & (price_history[-1] < ma):
        order_target(context.asset, 0)

    record(price=data.current(context.asset, 'price'),
           moving_average=ma)
    
    
sma_results = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=principal,
    bundle=bundle_name,
    data_frequency="daily",
)

# plot('Simple Moving Average - Tesla', sma_results, ['price', 'moving_average'])

/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:797: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
/Users/josephahn/opt/anaconda3/envs/ml4t/lib/python3.8/site-packages/empyrical/stats.py:706: RuntimeWarning: invalid value encountered in true_divide
  np.divide(


### Moving Average Crossover

In [8]:
SLOW_MA_PERIODS = 100
FAST_MA_PERIODS = 20

def initialize(context):
    context.days = 0
    context.asset = symbol(ticker)
    context.has_position = False
    
def handle_data(context, data):
    context.days += 1
    if context.days < SLOW_MA_PERIODS:
        return

    # calculate moving averages
    fast_ma = data.history(context.asset, 'price', bar_count=FAST_MA_PERIODS, frequency="1d").mean()
    slow_ma = data.history(context.asset, 'price', bar_count=SLOW_MA_PERIODS, frequency="1d").mean()

    # trading logic
    if (fast_ma > slow_ma) & (not context.has_position):
        order_percent(context.asset, 1.0)
        context.has_position = True
    elif (fast_ma < slow_ma) & (context.has_position):
        order_target(context.asset, 0)
        context.has_position = False

    record(price=data.current(context.asset, 'price'),
           fast_ma=fast_ma,
           slow_ma=slow_ma)

    
mac_results = run_algorithm(
    start=start,
    end=end,
    initialize=initialize,
    handle_data=handle_data,
    capital_base=principal,
    bundle=bundle_name,
    data_frequency="daily",
)

# plot('Moving Average Crossover Strategy - Tesla', mac_results, ['price', 'fast_ma', 'slow_ma'])

### Save results to CSV file

In [9]:
bah_results.to_csv(f'results/{ticker.lower()}/{start_year}/bah.csv', index=False)
dca_results.to_csv(f'results/{ticker.lower()}/{start_year}/dca.csv', index=False)
sma_results.to_csv(f'results/{ticker.lower()}/{start_year}/sma.csv', index=False)
mac_results.to_csv(f'results/{ticker.lower()}/{start_year}/mac.csv', index=False)